<a href="https://colab.research.google.com/github/olivermueller/aml4ta-2021/blob/main/Session_01/1_03_Conditional_word_counting_with_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


In [ ]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Install missing packages
!pip install pymysql

# <font color="#003660">Week 1: Basics of Natural Language Processing</font>

# <font color="#003660">Notebook 3: Conditional Word Counting with Spacy</font>

<center><br><img width=256 src="https://git.uni-paderborn.de/data.analytics.teaching/aml4ta-2020/-/raw/master/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you will be able to...</b><br><br>
        ... perform NLP preprocessing with Spacy.</font>
</div>
</center>
</p>

# Import packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `NLTK` is a leading platform for building Python programs to work with human language data.
- `Spacy` is a library for for industrial-strength natural language processing.
- `SQLAlchemy`, together with `pymysql`, allows to communicate with SQL databases.
- `getpass` provides function to safely enter passwords.
- `altair` is a visualization library based on the grammar of graphics.

In [ ]:
import pandas as pd
import nltk
import spacy
from sqlalchemy import create_engine
import getpass
import altair as alt

# Test drive Spacy

spaCy is an open-source library for Natural Language Processing (NLP) in Python. It helps you build NLP applications that process and understand large volumes of unstructured text. One of the main features of spaCy are linguistic annotations that give you insights into a text’s grammatical structure (e.g., word order, types of words, parts of speech, grammatical roles and relations).

At the center of spaCy is the processing pipeline, an object which is usually called `nlp`. The pipeline is build on top of a language-specific machine learning model and a set of handcrafted rules.

The nlp pipeline contains different components, each specialized for a specific NLP task. 

[More...](https://spacy.io/usage/spacy-101#whats-spacy)

<center><br><img src="
https://d33wubrfki0l68.cloudfront.net/3ad0582d97663a1272ffc4ccf09f1c5b335b17e9/7f49c/pipeline-fde48da9b43661abcdf62ab70a546d71.svg"/><br></center>

The following code creates a processing pipeline based on the `en_core_web_sm` model and assigns it to the variable `nlp`.

In [ ]:
nlp = spacy.load("en_core_web_sm")

Let's feed the `nlp` object with a simple sentence. When you process a text with the `nlp` object, spaCy creates a `doc` object. The `doc` lets you access information about the text in a structured way. 

In [ ]:
doc = nlp(u"Yesterday, I went to five pubs in Oxford.")

The first component of every spaCy pipeline is the `tokenizer`, which segments an unstructured text into words, punctuation, and so on. These `tokens` are the main contents of the `doc` object. [More...](https://spacy.io/usage/spacy-101#annotations-token)



In [ ]:
for token in doc:
  print(token.text)

The tokens contain many useful attributes. For example, we can access the `lemma` of each token. [More...](https://spacy.io/usage/linguistic-features#lemmatization
)



In [ ]:
for token in doc:
  print(token.text, "->", token.lemma_)

`Part-of-speech` tagging is the process of marking up a word in a text as corresponding to a particular part of speech, based on both its definition and its context. Examples of POS include words as nouns, verbs, adjectives, adverbs, etc. [More](https://spacy.io/usage/linguistic-features#pos-tagging
)

In [ ]:
for token in doc:
  print(token.text, "->", token.pos_, "(", spacy.explain(token.pos_), ")")

We can also recognize so-called named entities. A `named entity` is a “real-world object” that’s assigned a name – for example, a person, a country, or  a product. spaCy can recognize various types of named entities in a document. [More...](https://spacy.io/usage/linguistic-features#named-entities)

In [ ]:
for ent in doc.ents:
  print(ent.text, "->", ent.label_, "(", spacy.explain(ent.label_), ")")

# Load documents
This time, we load our data from a MySQL database. For security reasons, we don't store the database credentials here; please have a look at Panda to get them.

In [ ]:
# Get credentials
user = input("Username: ")
passwd = getpass.getpass("Password: ")
server = input("Server: ")
db = input("Database: ")

# Create an engine instance (SQLAlchemy)
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, passwd ,server, db))

# Define SQL query
sql_query = "SELECT * FROM BillboardLyrics"

# Query dataset (pandas)
corpus = pd.read_sql(sql=sql_query, con=engine)

# Sample
corpus.head()

In [ ]:
corpus.iloc[42]

# Preprocess documents with Spacy
Tokenization, stopword removal and lemmatization in one go.

In [ ]:
docs_prep = corpus.to_dict("records")
for i, entry in enumerate(docs_prep):
  if entry["Lyrics"]:
    doc = nlp(entry["Lyrics"])
    tokens_prep = [] 
    for token in doc:
      if token.is_alpha and not token.is_stop:
        tokens_prep.append(token.lemma_)
    entry["Lyrics_prep"] = tokens_prep
  else:
    entry["Lyrics_prep"] = []

In [ ]:
docs_prep[42]

# Conditional word counting
We seperately count words for each condition, that is, for each year. Unfortunately, this time we have to do this "by hand" and iterate through all docs and tokens and increase the token count for the respective condition.

In [ ]:
cfreq = nltk.ConditionalFreqDist()

for doc in docs_prep:
  for token in doc["Lyrics_prep"]:
    condition = doc["Year"]
    cfreq[condition][token] += 1

In [ ]:
cfreq[2010]

# Time series of word occurences

For all years between 1965 and 2015, get the frequency of the word "money".

In [ ]:
word = u"money"
years = range(1965,2016)
occurences = []
for year in years:
  occurences.append(cfreq[year][word])

Merge the years and the word occurcences in one dataframe.

In [ ]:
timeseries = pd.DataFrame(list(zip(years, occurences)),
              columns=['years','count'])
timeseries['years'] = pd.to_datetime(timeseries['years'], format='%Y')

Plot the time series.

In [ ]:
alt.Chart(timeseries).mark_line().encode(
    x='years',
    y='count'
).interactive()